# Downloading and maniupulating grib data from AWS Open Data Registry

In [1]:
import requests

url = "https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20250424/conus/hrrr.t06z.wrfsfcf00.grib2"
r = requests.get(url, stream=True)

with open("local_file.grib2", "wb") as f:
    for chunk in r.iter_content(chunk_size=8192): # 8192 bytes (8 KB) is good balance for RAM/speed
        f.write(chunk)

In [2]:
import pygrib

grbs = pygrib.open("local_file.grib2")

# List all messages
for grb in grbs:
    print(grb)   # prints metadata like name, level, units

1:Maximum/Composite radar reflectivity:dB (instant):lambert:atmosphere:level 0 -:fcst time 0 hrs:from 202504240600
2:3:3 (instant):lambert:cloudTop:level 0:fcst time 0 hrs:from 202504240600
3:201:201 (instant):lambert:atmosphere:level 0 -:fcst time 0 hrs:from 202504240600
4:Vertically-integrated liquid:kg m**-1 (instant):lambert:atmosphere:level 0 -:fcst time 0 hrs:from 202504240600
5:Visibility:m (instant):lambert:surface:level 0:fcst time 0 hrs:from 202504240600
6:Derived radar reflectivity:dB (instant):lambert:heightAboveGround:level 1000 m:fcst time 0 hrs:from 202504240600
7:Derived radar reflectivity:dB (instant):lambert:heightAboveGround:level 4000 m:fcst time 0 hrs:from 202504240600
8:Derived radar reflectivity:dB (instant):lambert:isothermal:level 263 K:fcst time 0 hrs:from 202504240600
9:Wind speed (gust):m s**-1 (instant):lambert:surface:level 0:fcst time 0 hrs:from 202504240600
10:U component of wind:m s**-1 (instant):lambert:isobaricInhPa:level 25000 Pa:fcst time 0 hrs:from

In [3]:
# Example: nearest grid point to 40N, -105W (Boulder, CO)
import numpy as np

lats, lons = grb.latlons()
dist = (lats-40)**2 + (lons+105)**2
i, j = np.unravel_index(dist.argmin(), dist.shape)

In [28]:
vars_levels = {
    "Surface pressure": {},
    "Forecast surface roughness": {},
    "Visible Beam Downward Solar Flux": {},
    "Visible Diffuse Downward Solar Flux": {},
    "2 metre temperature": {},
    "2 metre dewpoint temperature": {},
    "2 metre relative humidity": {},
    "10 metre U wind component": {},  # name includes 10m
    "10 metre V wind component": {},  # name includes 10m
    "U component of wind": {"level": 80},
    "V component of wind": {"level": 80},
}

for var, lvl_args in vars_levels.items():
    try:
        grb = grbs.select(name=var, **lvl_args)[0]
        data = grb.values
        print(f"{var}: {lvl_args}", data[i, j])
    except Exception as e:
        print(f"Error parsing {var}:", e)

Surface pressure: {} 83730.0
Forecast surface roughness: {} 0.0821
Visible Beam Downward Solar Flux: {} 0.0
Visible Diffuse Downward Solar Flux: {} 0.0
2 metre temperature: {} 284.3097229003906
2 metre dewpoint temperature: {} 277.72605895996094
2 metre relative humidity: {} 64.3
10 metre U wind component: {} -0.05168724060058594
10 metre V wind component: {} -2.5212268829345703
U component of wind: {'level': 80} -0.03038787841796875
V component of wind: {'level': 80} -2.567718505859375
